<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/Scrap_loading_page_with_selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.fQFyeHvS1a/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.vxFnVdPaSW/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.HHA8vqtPym/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as etc
import json
import time
import os
import requests
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium.webdriver.common.by import By

chrome_options = Options()


service = Service(executable_path=r'/usr/bin/chromedriver')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(service=service, options=chrome_options)
# url = ["https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-0/", "https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-1/", "https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-0/"]
# driver.get(url)

#Get All Links And category

In [14]:
# get all urls for loop
#Fetch the content from the front page
url_base = 'https://www.next.co.uk/children/the-character-shop'
content = requests.get(url_base).content
soup = BeautifulSoup(content, 'html.parser')

In [15]:
upper_data_links = []
down_data_links = []

up_links = soup.find_all('a', class_='hp-btn-invert')
d_links = soup.find_all('a', class_='baby-circular-swipe-img-con')
for link in up_links:
  full_link = urljoin(url_base, link['href'])
  category = link.text
  upper_data_links.append({'category': category, "links": full_link})
for link in d_links:
  full_link = urljoin(url_base, link['href'])
  category = link.text.strip()
  down_data_links.append({'category': category, "links": full_link})

upper_data_links.pop() # remove last one as named "Home" not category
all_links_category = upper_data_links + down_data_links
print(all_links_category[0]['links'])

https://www.next.co.uk/shop/gender-newborngirls-gender-newbornunisex-gender-oldergirls-gender-youngergirls-promotion-charactershop-0


#Scrap the urls

In [16]:
# for index, url in enumerate(all_links_category):
#   if index < 2:
#     driver.get(url)
#     items = []

#     last_height = driver.execute_script("return document.body.scrollHeight")
#     itemTargetCount = 100

#     while itemTargetCount > len(items):
#       driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#       time.sleep(1)
#       new_height = driver.execute_script("return document.body.scrollHeight")
#       if new_height == last_height:
#         break
#       last_height = new_height
#       elements = driver.find_elements(By.CLASS_NAME, 'produc-1mup83m')
#       # Create a list to store the src attributes
#       src_attributes = []

#       for element in elements:
#           # Find the <img> elements within each <a> element
#           # src = element.get_attribute("src")
#           href = element.get_attribute("href")
#           print(href)
#           # img_elements = element.find_elements(By.TAG_NAME, "img")

#       #     # Extract the src attribute from each <img> element
#       #     for img_element in img_elements:
#       #         src = img_element.get_attribute("src")
#       #         src_attributes.append(src)
#       # # Print the src attributes
#       # for src in src_attributes:
#       #     print("Image source:", src)
# driver.quit()


In [17]:
# # get individual page
# response_individual = requests.get("https://www.next.co.uk/g624209s2/d99927#d99927")
# soup = BeautifulSoup(response_individual.content, "html.parser")

# title_div = soup.find('div', class_='Title')
# title_h1 = title_div.find('h1')
# title = title_h1.text.strip()
# td_description = soup.find(id='ToneOfVoice')
# print(td_description)
driver.get('https://www.next.co.uk/style/ls298607/p99506#p99506')
try:
  composition = driver.find_element(By.ID, 'Composition')
  composition = composition.get_attribute("innerHTML")
except:
  composition = ""
try:
  description_head = driver.find_element(By.ID, 'ToneOfVoice')

    # Find all <li> elements inside the 'ToneOfVoice' element
  description_li = description_head.find_elements(By.TAG_NAME, 'li')

  # Iterate through <li> elements and print their text
  full_description = ""
  for element in description_li:
      full_description += element.get_attribute("innerHTML")
except:
   full_description = ""
description_feature = driver.find_element(By.ID, 'ToneOfVoice')
print(description_feature.get_attribute('outerHTML'))
# driver.quit()

<div id="ToneOfVoice">Featuring Encanto’s three Madrigal sisters – Mirabel, Isabella and Luisa on the long-sleeved top, with a contrasting all-over print on the pyjama bottoms. Official Disney licensed and sustainably made with supersoft BCI cotton – with a tree planted free with every order at brandthreads.com.</div>


In [18]:
# driver.get(all_links_category[0]['links'])
# items = []

# last_height = driver.execute_script("return document.body.scrollHeight")
# itemTargetCount = 100

# while itemTargetCount > len(items):
#   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#   time.sleep(1)
#   new_height = driver.execute_script("return document.body.scrollHeight")
#   if new_height == last_height:
#     break
#   last_height = new_height
#   elements = driver.find_elements(By.CLASS_NAME, 'produc-1mup83m')
#   # Create a list to store the src attributes
#   src_attributes = []

#   for index, element in enumerate(elements):
#       href = element.get_attribute("href")
#       # text_content = element.text
#       print(href)


#   #     # Extract the src attribute from each <img> element
#   #     for img_element in img_elements:
#   #         src = img_element.get_attribute("src")
#   #         src_attributes.append(src)
#   # # Print the src attributes
#   # for src in src_attributes:
#   #     print("Image source:", src)
# driver.quit()
